# Data Manipulation 

## Loading Data and Different Formats

Let's read the CSV file into a better format - Apache Arrow! 

In [ ]:
download("https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data-original",
         "data/auto.txt")

In [1]:
readlines("data/auto.txt")   # Just to see if it's there

406-element Array{String,1}:
 "18.0   8.   307.0      130.0      3504.      12.0   70.  1.\t\"chevrolet chevelle malibu\""
 "15.0   8.   350.0      165.0      3693.      11.5   70.  1.\t\"buick skylark 320\""
 "18.0   8.   318.0      150.0      3436.      11.0   70.  1.\t\"plymouth satellite\""
 "16.0   8.   304.0      150.0      3433.      12.0   70.  1.\t\"amc rebel sst\""
 "17.0   8.   302.0      140.0      3449.      10.5   70.  1.\t\"ford torino\""
 "15.0   8.   429.0      198.0      4341.      10.0   70.  1.\t\"ford galaxie 500\""
 "14.0   8.   454.0      220.0      4354.       9.0   70.  1.\t\"chevrolet impala\""
 "14.0   8.   440.0      215.0      4312.       8.5   70.  1.\t\"plymouth fury iii\""
 "14.0   8.   455.0      225.0      4425.      10.0   70.  1.\t\"pontiac catalina\""
 "15.0   8.   390.0      190.0      3850.       8.5   70.  1.\t\"amc ambassador dpl\""
 "NA     4.   133.0      115.0      3090.      17.5   70.  2.\t\"citroen ds-21 pallas\""
 "NA     8.   350.0      

In [2]:
using CSV
using DataFrames

In [3]:
df = CSV.File("data/auto.txt",
               delim=' ',
               ignorerepeated=true,
               header=[:mpg, :cylinders, :displacement, :horsepower,
                       :weight, :acceleration, :year, :origin, :name],
               missingstring="NA") |>
      DataFrame

┌ Warning: thread = 1 warning: parsed expected 9 columns, but didn't reach end of line around data row: 1. Ignoring any extra columns on this row
└ @ CSV /Users/nam/.julia/packages/CSV/YUbbG/src/file.jl:604


,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,"1.\t""chevrolet"


That didn't work well. So, let's try to fix this. 

In [4]:
raw_str = read("data/auto.txt", String);

In [5]:
str_no_tab = replace(raw_str, '\t'=>' ');

In [6]:
io = IOBuffer(str_no_tab)

IOBuffer(data=UInt8[...], readable=true, writable=false, seekable=true, append=false, size=32149, maxsize=Inf, ptr=1, mark=-1)

In [7]:
df = CSV.File(io,
               delim=' ',
               ignorerepeated=true,
               header=[:mpg, :cylinders, :displacement, :horsepower,
                       :weight, :acceleration, :year, :origin, :name],
               missingstring="NA") |>
      DataFrame

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0


## Missing Values 

In [8]:
sum(count(ismissing, col) for col in eachcol(df))

14

In [9]:
count(ismissing, Matrix(df)) # much cleaner 

14

In [10]:
mapcols(x -> count(ismissing, x), df)   
# take a column and send it to the function 
# this is more descriptive, of course 

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,8,0,0,6,0,0,0,0


## Adding a Column based on Data 

Let's take the name and take it's first part for "brand". 

In [11]:
df.name

406-element Array{String,1}:
 "chevrolet chevelle malibu"
 "buick skylark 320"
 "plymouth satellite"
 "amc rebel sst"
 "ford torino"
 "ford galaxie 500"
 "chevrolet impala"
 "plymouth fury iii"
 "pontiac catalina"
 "amc ambassador dpl"
 "citroen ds-21 pallas"
 "chevrolet chevelle concours (sw)"
 "ford torino (sw)"
 ⋮
 "buick century limited"
 "oldsmobile cutlass ciera (diesel)"
 "chrysler lebaron medallion"
 "ford granada l"
 "toyota celica gt"
 "dodge charger 2.2"
 "chevrolet camaro"
 "ford mustang gl"
 "vw pickup"
 "dodge rampage"
 "ford ranger"
 "chevy s-10"

In [13]:
split.(df.name)
df.brand = first.(split.(df.name))

406-element Array{SubString{String},1}:
 "chevrolet"
 "buick"
 "plymouth"
 "amc"
 "ford"
 "ford"
 "chevrolet"
 "plymouth"
 "pontiac"
 "amc"
 "citroen"
 "chevrolet"
 "ford"
 ⋮
 "buick"
 "oldsmobile"
 "chrysler"
 "ford"
 "toyota"
 "dodge"
 "chevrolet"
 "ford"
 "vw"
 "dodge"
 "ford"
 "chevy"

In [15]:
df[1:10, ["name", "brand"]]

,name,brand
,String,SubStri…
1,chevrolet chevelle malibu,chevrolet
2,buick skylark 320,buick
3,plymouth satellite,plymouth
4,amc rebel sst,amc
5,ford torino,ford
6,ford galaxie 500,ford
7,chevrolet impala,chevrolet
8,plymouth fury iii,plymouth
9,pontiac catalina,pontiac


In [16]:
size(df)

(406, 10)

In [17]:
df2 = dropmissing(df);

In [18]:
size(df2)

(392, 10)

In [19]:
mapcols(x -> count(ismissing, x), df2)   

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,0,0,0,0,0,0,0,0


In [20]:
df2[df2.brand .== "saab", :]  # index-style syntax

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,25.0,4.0,104.0,95.0,2375.0,17.5,70.0,2.0
2,24.0,4.0,121.0,110.0,2660.0,14.0,73.0,2.0
3,25.0,4.0,121.0,115.0,2671.0,13.5,75.0,2.0
4,21.6,4.0,121.0,115.0,2795.0,15.7,78.0,2.0


In [21]:
# same as above 
filter(row -> row.brand == "saab", df2)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,25.0,4.0,104.0,95.0,2375.0,17.5,70.0,2.0
2,24.0,4.0,121.0,110.0,2660.0,14.0,73.0,2.0
3,25.0,4.0,121.0,115.0,2671.0,13.5,75.0,2.0
4,21.6,4.0,121.0,115.0,2795.0,15.7,78.0,2.0


## Saving For Future Use 

In [22]:
CSV.write("data/auto-cleaned.csv", df2)

"data/auto-cleaned.csv"

We will also use Arrow to save our data. Highly recommended for storing all data. Read more here: https://arrow.apache.org/

In [ ]:
] add Arrow 

In [23]:
using Arrow

In [24]:
Arrow.write("data/auto-cleaned.arrow", df2)

"data/auto-cleaned.arrow"

## Further Data Manipulations 

In [25]:
df2 = Arrow.Table("data/auto-cleaned.arrow") |> DataFrame

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0


In [26]:
grouped_brands = groupby(df2, :brand)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0
2,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0
3,15.0,8.0,400.0,150.0,3761.0,9.5,70.0,1.0
4,28.0,4.0,140.0,90.0,2264.0,15.5,71.0,1.0
5,17.0,6.0,250.0,100.0,3329.0,15.5,71.0,1.0
6,14.0,8.0,350.0,165.0,4209.0,12.0,71.0,1.0
7,22.0,4.0,140.0,72.0,2408.0,19.0,71.0,1.0
8,20.0,4.0,140.0,90.0,2408.0,19.5,72.0,1.0
9,13.0,8.0,350.0,165.0,4274.0,12.0,72.0,1.0


In [29]:
grouped_brands[ ("ford", )]       

# Notice the trailing comma!

# This has to be a tuple because we could've grouped 
#  by more than one cols 

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0
2,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0
3,21.0,6.0,200.0,85.0,2587.0,16.0,70.0,1.0
4,10.0,8.0,360.0,215.0,4615.0,14.0,70.0,1.0
5,19.0,6.0,250.0,88.0,3302.0,15.5,71.0,1.0
6,14.0,8.0,351.0,153.0,4154.0,13.5,71.0,1.0
7,13.0,8.0,400.0,170.0,4746.0,12.0,71.0,1.0
8,18.0,6.0,250.0,88.0,3139.0,14.5,71.0,1.0
9,21.0,4.0,122.0,86.0,2226.0,16.5,72.0,1.0


In [30]:
using Statistics

In [31]:
brand_mpg = combine(grouped_brands, :mpg => mean)

,brand,mpg_mean
,String,Float64
1,chevrolet,20.4721
2,buick,19.1824
3,plymouth,21.7032
4,amc,18.0704
5,ford,19.475
6,pontiac,20.0125
7,dodge,22.0607
8,toyota,28.372
9,datsun,31.113


In [32]:
ENV["LINES"] = 10    # I find the large number of rows annoying 

10

In [33]:
brand_mpg = combine(grouped_brands, :mpg => mean)

# takes the mean column from each group and 
# passes it to the mean function 
# creates a 'group_name' -> 'aggregate' 

,brand,mpg_mean
,String,Float64
1,chevrolet,20.4721
2,buick,19.1824
3,plymouth,21.7032
4,amc,18.0704
5,ford,19.475
6,pontiac,20.0125
7,dodge,22.0607
8,toyota,28.372
9,datsun,31.113


We can also sort it based on the mean. 

In [34]:
sort!(brand_mpg, :mpg_mean, rev=true)  # in-place sort

,brand,mpg_mean
,String,Float64
1,vw,39.0167
2,nissan,36.0
3,triumph,35.0
4,honda,33.7615
5,datsun,31.113
6,mazda,30.86
7,subaru,30.525
8,vokswagen,29.8
9,renault,29.6667


## Piping and Data Wrangling 

There are usually many stages through which we want our data to go through. These are "assembled" or chained together. Piping makes this very readable for us. 

We want to read the data from our original data set, group it by brand and then use combine to calculate aggregates on it. 

In [35]:
df.origin  # where the brand is from 

406-element Array{Float64,1}:
 1.0
 1.0
 1.0
 ⋮
 1.0
 1.0

In [36]:
orig_brand = groupby(df, :brand)
orig_brand = combine(orig_brand, 
                     :origin => x -> length(unique(x)))

# notice the name of the resulting column

,brand,origin_function
,SubStri…,Int64
1,chevrolet,1
2,buick,1
3,plymouth,1
4,amc,1
5,ford,1
6,pontiac,1
7,citroen,1
8,dodge,1
9,toyota,1


In [ ]:
# We can make this prettier and less error-prone. 

In [ ]:
] add Pipe 

In [37]:
using Pipe

In [38]:
orig_brand = @pipe df |>  
            groupby(_, :brand)  |>  
            combine(_, :origin => x -> length(unique(x)))

,brand,origin_function
,SubStri…,Int64
1,chevrolet,1
2,buick,1
3,plymouth,1
4,amc,1
5,ford,1
6,pontiac,1
7,citroen,1
8,dodge,1
9,toyota,1


In [39]:
# each brand should have an origin of 1. Let's verify. 
extrema(orig_brand.origin_function)

(1, 1)

We can also count the number of data points for each brand. 

In [40]:
origin_brand2 = @pipe df |>
                groupby(_, [:origin, :brand]) |>
                combine(_, nrow)  |> 
                sort(_, :nrow, rev=true)

,origin,brand,nrow
,Float64,SubStri…,Int64
1,1.0,ford,53
2,1.0,chevrolet,44
3,1.0,plymouth,32
4,1.0,amc,29
5,1.0,dodge,28
6,3.0,toyota,25
7,3.0,datsun,23
8,1.0,buick,17
9,1.0,pontiac,16


In [41]:
origin_vs_brand = unstack(origin_brand2, 
                        :brand, :origin, :nrow)

,brand,1.0,3.0,2.0
,SubStri…,Int64?,Int64?,Int64?
1,ford,53,missing,missing
2,chevrolet,44,missing,missing
3,plymouth,32,missing,missing
4,amc,29,missing,missing
5,dodge,28,missing,missing
6,toyota,missing,25,missing
7,datsun,missing,23,missing
8,buick,17,missing,missing
9,pontiac,16,missing,missing


In [42]:
coalesce.(origin_vs_brand, 0)   
# replace missing values with 0 because 
# that makes sense here 

,brand,1.0,3.0,2.0
,SubStri…,Int64,Int64,Int64
1,ford,53,0,0
2,chevrolet,44,0,0
3,plymouth,32,0,0
4,amc,29,0,0
5,dodge,28,0,0
6,toyota,0,25,0
7,datsun,0,23,0
8,buick,17,0,0
9,pontiac,16,0,0
